# Simple Data

The goal of this Notebook is simplify the data to be directly readable, as done in exp001 (this is very similar).

In [1]:
INHABITANTS_GERMANY = 83.2E6 # https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Bevoelkerungsstand/_inhalt.html

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
corona_data = pd.read_csv("../dat/CoronaData.csv", parse_dates=["date"])
weather_data = pd.read_csv("../dat/WeatherData.csv", parse_dates=["date"])

In [4]:
corona_data.head()

,Unnamed: 0,id_county,name_county,id_state,name_state,cases,deaths,date,cum_cases,cum_deaths
0,0,1001.0,SK Flensburg,1.0,Schleswig-Holstein,0.0,0.0,2020-01-02,0.0,0.0
1,1,1001.0,SK Flensburg,1.0,Schleswig-Holstein,0.0,0.0,2020-01-03,0.0,0.0
2,2,1001.0,SK Flensburg,1.0,Schleswig-Holstein,0.0,0.0,2020-01-04,0.0,0.0
3,3,1001.0,SK Flensburg,1.0,Schleswig-Holstein,0.0,0.0,2020-01-05,0.0,0.0
4,4,1001.0,SK Flensburg,1.0,Schleswig-Holstein,0.0,0.0,2020-01-06,0.0,0.0


In [5]:
weather_data.head()

,date,air temperature
0,2019-01-01 01:00:00,6.0400
1,2019-01-01 02:00:00,6.0350
2,2019-01-01 03:00:00,5.9325
3,2019-01-01 04:00:00,5.8650
4,2019-01-01 05:00:00,5.8325


## Simplify data

For this first experiment, we will simplify the data by reducing it to daily precision and germanywide data.

### Corona Data

In [6]:
corona_data_simple = corona_data.groupby("date").sum()[["cases", "deaths"]]
corona_data_simple

,cases,deaths
date,,
2020-01-02,1.0,0.0
2020-01-03,0.0,0.0
2020-01-04,0.0,0.0
2020-01-05,0.0,0.0
2020-01-06,0.0,0.0
...,...,...
2022-02-02,229055.0,32.0
2022-02-03,223339.0,15.0
2022-02-04,180107.0,5.0


In order to be more useful, the data has to be smoothed over seven days

In [7]:
cases = corona_data_simple.cases.to_numpy()
cases_smoothed = np.convolve(cases, np.ones(7), mode="same")
deaths = corona_data_simple.deaths.to_numpy()
deaths_smoothed = np.convolve(deaths, np.ones(7), mode="same")

corona_data_simple["cases_smoothed"] = cases_smoothed / 7
corona_data_simple["deaths_smoothed"] = deaths_smoothed / 7

Now we can calculate the case fatality rate. Since the values are not reliable in the beginning, we will set $cfr = 0$ if there have been few (e.g. 2) new cases that day (this should only occur before the first wave).

In [8]:
offset = 14

In [9]:
cases_smoothed_modified = corona_data_simple["cases_smoothed"].apply(lambda x: x if x > 2 else np.inf)
cfr = corona_data_simple["deaths_smoothed"][offset:] / cases_smoothed_modified[:-offset]

In [10]:
corona_data_simple["cfr"] = cfr

In [11]:
corona = corona_data_simple.fillna(method="ffill").fillna(method="bfill")

In [12]:
corona

,cases,deaths,cases_smoothed,deaths_smoothed,cfr
date,,,,,
2020-01-02,1.0,0.0,0.142857,0.000000,0.000000
2020-01-03,0.0,0.0,0.142857,0.000000,0.000000
2020-01-04,0.0,0.0,0.142857,0.000000,0.000000
2020-01-05,0.0,0.0,0.142857,0.000000,0.000000
2020-01-06,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...
2022-02-02,229055.0,32.0,166399.714286,19.142857,0.000379
2022-02-03,223339.0,15.0,156089.285714,16.857143,0.000379
2022-02-04,180107.0,5.0,133983.571429,11.142857,0.000379


### Weather data

In [13]:
weather_data

,date,air temperature
0,2019-01-01 01:00:00,6.040000
1,2019-01-01 02:00:00,6.035000
2,2019-01-01 03:00:00,5.932500
3,2019-01-01 04:00:00,5.865000
4,2019-01-01 05:00:00,5.832500
...,...,...
27162,2022-02-05 19:00:00,3.542105
27163,2022-02-05 20:00:00,3.410526
27164,2022-02-05 21:00:00,3.497368
27165,2022-02-05 22:00:00,3.431579


Filter to only daytime

In [14]:
weather_data = weather_data[(weather_data.date.apply(lambda x: x.hour) >= 6) & (weather_data.date.apply(lambda x: x.hour) <= 20)]
weather_data

,date,air temperature
5,2019-01-01 06:00:00,5.842500
6,2019-01-01 07:00:00,5.777500
7,2019-01-01 08:00:00,5.777500
8,2019-01-01 09:00:00,5.760000
9,2019-01-01 10:00:00,5.925000
...,...,...
27159,2022-02-05 16:00:00,4.352632
27160,2022-02-05 17:00:00,3.897368
27161,2022-02-05 18:00:00,3.571053
27162,2022-02-05 19:00:00,3.542105


In [15]:
weather = weather_data.set_index("date").groupby(lambda x: f"{x.year}-{x.month:02d}-{x.day:02d}").mean()
weather.index.names = ["date"]

In [16]:
num_convolve = 5

weather["air temperature"] = np.convolve(
    weather["air temperature"], np.ones(num_convolve), "same"
) / num_convolve

In [17]:
weather.head()

,air temperature
date,
2019-01-01,1.310200
2019-01-02,1.579467
2019-01-03,2.436467
2019-01-04,2.057633
2019-01-05,2.503900


## Combining the data

In [18]:
weather.head()

,air temperature
date,
2019-01-01,1.310200
2019-01-02,1.579467
2019-01-03,2.436467
2019-01-04,2.057633
2019-01-05,2.503900


In [19]:
corona.head()

,cases,deaths,cases_smoothed,deaths_smoothed,cfr
date,,,,,
2020-01-02,1.0,0.0,0.142857,0.0,0.0
2020-01-03,0.0,0.0,0.142857,0.0,0.0
2020-01-04,0.0,0.0,0.142857,0.0,0.0
2020-01-05,0.0,0.0,0.142857,0.0,0.0
2020-01-06,0.0,0.0,0.000000,0.0,0.0


In [20]:
combined = corona.merge(weather, how="inner", left_index=True, right_index=True)

In [21]:
combined.head()

,cases,deaths,cases_smoothed,deaths_smoothed,cfr,air temperature
date,,,,,,
2020-01-02,1.0,0.0,0.142857,0.0,0.0,2.519630
2020-01-03,0.0,0.0,0.142857,0.0,0.0,2.491579
2020-01-04,0.0,0.0,0.142857,0.0,0.0,2.989614
2020-01-05,0.0,0.0,0.142857,0.0,0.0,3.619053
2020-01-06,0.0,0.0,0.000000,0.0,0.0,3.735754


In [22]:
combined.to_csv("../dat/SimpleCombinedData.csv")